<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/GPT%E8%AF%A6%E8%A7%A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. GPT基本思想

&emsp;&emsp;先通过无标签的文本去训练生成语言模型，再跟据具体的NLP任务（如文本蕴含、QA、文本分类等），来通过有标签的数据对模型进行fine-tuning。具体来说提出了半监督的方法，即结合了无监督的预训练和有监督的fine-tuning。采用两阶段训练。首先在未标记数据集上训练语言模型来学习神经网络模型的初始参数。随后，使用NLP任务 中的有标签数据将这些参数进行微调，以适应当前任务。

## 2. GPT模型结构

GPT模型使用了多层的单向Transformer结构。

<img src="https://pic3.zhimg.com/80/v2-fb7be8467a0231607f3f2e4ace92534e_720w.jpg" width="500" />

&emsp;&emsp;GPT使用Transformer 的Decoder结构，并对Decoder进行了一些改动，原本的Decoder包含两个Multi-Head Attention结构，GPT只保留了Mask Multi-Head Attention结构。


<img src="https://pic4.zhimg.com/80/v2-7fa8e82f59c789e525d94fd2150941bb_720w.jpg" width="300" />

&emsp;&emsp;GPT使用句子序列预测下一个单词，因此采用Mask Multi-Head Attention对单词的下文进行遮挡，防止信息泄露。

&emsp;&emsp;Mask 操作是在 Self-Attention 进行 Softmax 之前进行的，具体做法是将要 Mask 的位置用一个无穷小的数替换 -inf，然后再 Softmax，如下图所示。

<img src="https://lh3.googleusercontent.com/proxy/zGqnhot2_WQe4ALcKov7uaeNWef25h5sj8DWkS_pge7mBSj8Pf8R30Dz_ivt9FfSihxmbHSFzn-92RsTLTyAbByXLZwSTTF1aklwnBLbl5A9DqgtKe7ABlnP2PPBCX7MpNrEF3VvNeGb6w2N3fyqzEJnTNF23Jeh5kUb2L1T_Opxjt6oAAz3xnDG7Fje0nNk6vvBQu3ScI8B2t0tW7qvumwPBlboCbv53ifqAg" width="300" />


<img src="https://lh3.googleusercontent.com/yO_7X2nsbJKzizQUJN--2WMuujUisKgpWeKu6_nFv--GMLDQHAWtTkRY4-28UC7jwTK1xg=s170" width="300" />

可以看到，经过 Mask 和 Softmax 之后，当 GPT 根据单词 A 预测单词 B 时，只能使用单词 A 的信息，根据 [A, B] 预测单词 C 时只能使用单词 A, B 的信息。这样就可以防止信息泄露。


下图是 GPT 整体模型图，其中包含了 12 个 Decoder。

<img src="https://cdn.zhuanzhi.ai/images/wx/7f228cd4d90f5868eb836bf02f4a1e67" width="500" />


## 3. GPT预训练
&emsp;&emsp;GPT模型的训练分为两个阶段：

### 3.1 无监督的预训练
&emsp;&emsp;对于无标签的文本$U=\{u_1,\ldots,u_n\}$，最大化语言模型的极大似然函数：

<img src="https://pic1.zhimg.com/80/v2-ba7eaac4fff4c71d8bf93715ad742490_720w.png" width="500" />

可以看到GPT是一个单向模型，其输入用$h_0$表示：
$h_0=UW_e+W_p$

$W_p$是单词的位置的Embedding,$W_e$是单词的embedding。用 voc 表示词汇表大小，pos 表示最长的句子长度，dim 表示 Embedding 维度，则 $W_p$ 是一个 pos×dim 的矩阵，$W_e$ 是一个 voc×dim 的矩阵。

得到输入 $h_0$ 之后，需要将 $h_0$ 依次传入 GPT 的所有 Transformer Decoder 里，最终得到 $h_t$。

$$
h_t=transformer_block(h_{l-1} \in [1,t])
$$
再用$h_t$预测下个单词的概率。

$$
P(u)=softmax(h_tW_e^T)
$$

###3.2 有监督的fine-tuning

&emsp;&emsp;GPT 经过预训练之后，会针对具体的下游任务对模型进行微调。微调的过程采用的是有监督学习，训练样本包括单词序列 [x1, x2, ..., xm] 和 类标 y。GPT 微调的过程中根据单词序列 [x1, x2, ..., xm] 预测类标 y。

<img src="https://pic3.zhimg.com/80/v2-314a77daec4ff0e2ac854a2a189113ae_720w.png" width="300" />

$W_y$表示预测输出时的参数，微调时候需要最大化以下函数。

<img src="https://pic3.zhimg.com/80/v2-813e9455e99090d2b0b704df83f8d51e_720w.jpg" width="300" />

最后，将两阶段的目标函数通过超参$\lambda$ 相加训练整个模型：

<img src="https://pic3.zhimg.com/80/v2-b8b38bc7b2a00da53d2eee5297b27bd2_720w.png" width="300" />


## 4. 具体任务的模型微调

<img src="https://pic4.zhimg.com/80/v2-4416df20d7bcb43d67a575afc3e0d9cf_720w.jpg" width="500" />

对于文本分类，只需要在预训练模型上微调。对于QA任务或者文本蕴含，因为预训练模型是在连续序列上训练，需要做一些调整，修改输入结构，将输入转化为有序序列输入。\
1）文本蕴含 ：将前提 [公式] 和假设 [公式] 序列拼接，中间用($)符号来分隔两个序列。\
2）文本相似度：分别将两个序列输入，通过模型输出两个序列的特征向量，再逐元素相加输入线性层。 \
3）问答和常识推理：给定上下文文本 [公式] ，问题 [公式] ，一组可能的候选答案 [公式] ，将上下文文本、问题以及每个候选答案拼接起来，得到这样一个序列 [公式] ，再将该序列输入预训练模型，经softmax层得到候选答案的概率分布。

##5. GPT与BERT模型的区别

* 预训练

&emsp;&emsp;GPT 预训练的方式和传统的语言模型一样，通过上文，预测下一个单词；BERT 预训练的方式是使用 Mask LM，可以同时通过上文和下文预测单词。例如给定一个句子 [u1, u2, ..., un]，GPT 在预测单词 ui 的时候只会利用 [u1, u2, ..., u(i-1)] 的信息，而 BERT 会同时利用 [u1, u2, ..., u(i-1), u(i+1), ..., un] 的信息。如下图所示。

<img src="https://i.redd.it/cw04e9546gv11.png" width="500" />

* 模型效果

&emsp;&emsp; GPT 因为采用了传统语言模型所以更加适合用于自然语言生成类的任务 (NLG)，因为这些任务通常是根据当前信息生成下一刻的信息。而 BERT 更适合用于自然语言理解任务 (NLU)。

* 模型结构
&emsp;&emsp;GPT 采用了 Transformer 的 Decoder，而 BERT 采用了 Transformer 的 Encoder。GPT 使用 Decoder 中的 Mask Multi-Head Attention 结构，在使用 [u1, u2, ..., u(i-1)] 预测单词 ui 的时候，会将 ui 之后的单词 Mask 掉。











